In [ ]:
!pip3 install matplotlib
!pip3 install pandas
!pip3 install sklearn
!pip3 install tensorflow

In [ ]:
import os
import random
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import numpy as np
import matplotlib.pylab as plt
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import mnist
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import DenseNet121


In [ ]:
df = pd.read_csv("binary_data.csv")

df['id_code'] = df['id_code'].astype(str) + '.png'

X = df["id_code"]
y = df["diagnosis"]


df['filepath'] = df.apply(lambda row: os.path.join("ADNI_IMAGES",
                                                   'AD' if row['diagnosis'] == 4 else 'CN',
                                                   row['id_code']), axis=1)

train_df, temp_df = train_test_split(df, test_size=0.4, stratify=df['diagnosis'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['diagnosis'], random_state=42)

train_df['diagnosis'] = train_df['diagnosis'].map({0: 'CN', 4: 'AD'})
val_df['diagnosis'] = val_df['diagnosis'].map({0: 'CN', 4: 'AD'})
test_df['diagnosis'] = test_df['diagnosis'].map({0: 'CN', 4: 'AD'})




datagen = ImageDataGenerator(rescale=1./255)


train_data = datagen.flow_from_dataframe(train_df, x_col='filepath', y_col='diagnosis',
                                         target_size=(224, 224), batch_size=32,
                                         class_mode='binary', shuffle=True, classes=['CN', 'AD'],)

val_data = datagen.flow_from_dataframe(val_df, x_col='filepath', y_col='diagnosis',
                                       target_size=(224, 224), batch_size=32,
                                       class_mode='binary', shuffle=False, classes=['CN', 'AD'],)

test_data = datagen.flow_from_dataframe(test_df, x_col='filepath', y_col='diagnosis',
                                        target_size=(224, 224), batch_size=32,
                                        class_mode='binary', shuffle=False, classes=['CN', 'AD'],)

print(train_data.class_indices)



In [ ]:
#Base model with more layers
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(256, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2, 2)),

    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')  # <--- binary output (Alzheimer's vs Normal)
])



model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
model.fit(
    train_data,
    validation_data=val_data,
    epochs=20
)


In [ ]:
#Original base model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),

    Dense(1, activation='sigmoid')  # <--- binary output (Alzheimer's vs Normal)
])


model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(
    train_data,
    validation_data=val_data,
    epochs=20)

In [ ]:
# Transfer learning model with VGG16
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

base_model.trainable = False

model = Sequential([
    base_model, 
    GlobalAveragePooling2D(),  
    Dense(128, activation='relu'),  
    #Dropout(0.5),  
    Dense(1, activation='sigmoid')  
])

model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

model.fit(
    train_data,
    validation_data=val_data,
    epochs=20
)

In [ ]:
# Transfer learning model with ResNet
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  

model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  
])

# Compile the model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Train the model
model.fit(
    train_data,
    validation_data=val_data,
    epochs=20
)

